# DQN for Atari Pong

Clean implementation of Deep Q-Network (DQN) for PongNoFrameskip-v4

**Run cells in order:**
1. Imports
2. Wrappers
3. Model
4. Replay Buffer
5. DQN Agent
6. Training

In [1]:
# Cell 1: Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import cv2
import random
from collections import deque, namedtuple
import gymnasium as gym
from gymnasium import Wrapper, ObservationWrapper, RewardWrapper
from tqdm import tqdm
import os

cv2.ocl.setUseOpenCL(False)  # Disable OpenCL for reproducibility

print("✅ Imports complete")

✅ Imports complete


In [2]:
# Cell 2: Atari Environment Wrappers
# Adapted from OpenAI Baselines for gymnasium

class NoopResetEnv(Wrapper):
    """Sample initial states by taking random number of no-ops on reset."""
    def __init__(self, env, noop_max=30):
        super().__init__(env)
        self.noop_max = noop_max
        self.noop_action = 0
    
    def reset(self, **kwargs):
        obs, info = self.env.reset(**kwargs)
        if hasattr(self.env.unwrapped, 'np_random'):
            noops = self.env.unwrapped.np_random.integers(1, self.noop_max + 1)
        else:
            noops = random.randint(1, self.noop_max)
        
        for _ in range(noops):
            obs, _, terminated, truncated, _ = self.env.step(self.noop_action)
            if terminated or truncated:
                obs, info = self.env.reset(**kwargs)
        return obs, info


class MaxAndSkipEnv(Wrapper):
    """Return only every `skip`-th frame, max-pooling over last 2 frames."""
    def __init__(self, env, skip=4):
        super().__init__(env)
        self._obs_buffer = np.zeros((2,) + env.observation_space.shape, dtype=np.uint8)
        self._skip = skip
    
    def step(self, action):
        total_reward = 0.0
        terminated = False
        truncated = False
        info = {}
        
        for i in range(self._skip):
            obs, reward, term, trunc, inf = self.env.step(action)
            if i == self._skip - 2:
                self._obs_buffer[0] = obs
            if i == self._skip - 1:
                self._obs_buffer[1] = obs
            total_reward += reward
            terminated = terminated or term
            truncated = truncated or trunc
            info.update(inf)
            if terminated or truncated:
                break
        
        max_frame = self._obs_buffer.max(axis=0)
        return max_frame, total_reward, terminated, truncated, info


class WarpFrame(ObservationWrapper):
    """Warp frames to 84x84 grayscale as done in the Nature paper."""
    def __init__(self, env):
        super().__init__(env)
        self.width = 84
        self.height = 84
        self.observation_space = gym.spaces.Box(
            low=0, high=255, shape=(self.height, self.width), dtype=np.uint8
        )
    
    def observation(self, frame):
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        frame = cv2.resize(frame, (self.width, self.height), interpolation=cv2.INTER_AREA)
        return frame


class EpisodicLifeEnv(Wrapper):
    """Make end-of-life == end-of-episode, but only reset on true game over."""
    def __init__(self, env):
        super().__init__(env)
        self.lives = 0
        self.was_real_done = True
    
    def step(self, action):
        obs, reward, terminated, truncated, info = self.env.step(action)
        self.was_real_done = terminated or truncated
        
        # Check current lives
        lives = info.get('lives', self.lives)
        if lives < self.lives and lives > 0:
            terminated = True
        self.lives = lives
        
        return obs, reward, terminated, truncated, info
    
    def reset(self, **kwargs):
        if self.was_real_done:
            obs, info = self.env.reset(**kwargs)
        else:
            obs, _, _, _, info = self.env.step(0)  # No-op step
        self.lives = info.get('lives', 0)
        return obs, info


class ClipRewardEnv(RewardWrapper):
    """Bin reward to {+1, 0, -1} by its sign."""
    def reward(self, reward):
        return np.sign(reward)


class FrameStack(Wrapper):
    """Stack k last frames. Returns lazy array, which is much more memory efficient."""
    def __init__(self, env, k=4):
        super().__init__(env)
        self.k = k
        self.frames = deque([], maxlen=k)
        obs_shape = env.observation_space.shape
        self.observation_space = gym.spaces.Box(
            low=0, high=255, shape=(k, *obs_shape), dtype=np.uint8
        )
    
    def reset(self, **kwargs):
        obs, info = self.env.reset(**kwargs)
        for _ in range(self.k):
            self.frames.append(obs)
        return self._get_obs(), info
    
    def step(self, action):
        obs, reward, terminated, truncated, info = self.env.step(action)
        self.frames.append(obs)
        return self._get_obs(), reward, terminated, truncated, info
    
    def _get_obs(self):
        assert len(self.frames) == self.k
        return np.stack(list(self.frames), axis=0)


def make_atari_env(env_id, max_episode_steps=None):
    """Create and configure Atari environment."""
    env = gym.make(env_id)
    if max_episode_steps:
        env = gym.wrappers.TimeLimit(env, max_episode_steps=max_episode_steps)
    
    # Apply standard Atari wrappers
    env = NoopResetEnv(env, noop_max=30)
    env = MaxAndSkipEnv(env, skip=4)
    env = WarpFrame(env)
    env = EpisodicLifeEnv(env)
    env = ClipRewardEnv(env)
    env = FrameStack(env, k=4)
    
    return env


print("✅ Wrappers defined")

✅ Wrappers defined


In [3]:
# Cell 3: DQN Model (CNN)

class DQNModel(nn.Module):
    """
    DQN CNN architecture from Nature paper.
    Input: (batch, 4, 84, 84) - 4 stacked grayscale frames
    Output: (batch, n_actions) - Q-values for each action
    """
    def __init__(self, n_actions):
        super(DQNModel, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(4, 32, kernel_size=8, stride=4)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        
        # Fully connected layers
        # After conv layers: (64, 7, 7) = 3136 features
        self.fc1 = nn.Linear(3136, 512)
        self.fc2 = nn.Linear(512, n_actions)
    
    def forward(self, x):
        # Normalize input to [0, 1]
        x = x.float() / 255.0
        
        # Convolutional layers with ReLU
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        
        # Flatten
        x = x.view(x.size(0), -1)
        
        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x


print("✅ Model defined")

✅ Model defined


In [4]:
# Cell 4: Experience Replay Buffer

Experience = namedtuple('Experience', ['state', 'action', 'reward', 'next_state', 'done'])

class ReplayBuffer:
    """
    Experience replay buffer for DQN.
    Stores transitions and samples random batches for training.
    """
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)
        self.capacity = capacity
    
    def push(self, state, action, reward, next_state, done):
        """Add a transition to the buffer."""
        experience = Experience(state, action, reward, next_state, done)
        self.buffer.append(experience)
    
    def sample(self, batch_size):
        """Sample a random batch of transitions."""
        batch = random.sample(self.buffer, batch_size)
        
        # Unpack and convert to tensors
        states = torch.FloatTensor(np.array([e.state for e in batch]))
        actions = torch.LongTensor([e.action for e in batch])
        rewards = torch.FloatTensor([e.reward for e in batch])
        next_states = torch.FloatTensor(np.array([e.next_state for e in batch]))
        dones = torch.FloatTensor([e.done for e in batch])
        
        return states, actions, rewards, next_states, dones
    
    def __len__(self):
        return len(self.buffer)


print("✅ Replay Buffer defined")

✅ Replay Buffer defined


In [5]:
# Cell 5: DQN Agent

class DQNAgent:
    """
    DQN Agent with experience replay and target network.
    """
    def __init__(
        self,
        state_shape,
        n_actions,
        device='cuda',
        lr=0.0000625,  # Learning rate from Nature paper
        gamma=0.99,
        epsilon_start=1.0,
        epsilon_end=0.1,
        epsilon_decay=1000000,  # Decay over 1M steps
        target_update_freq=10000,  # Update target network every 10k steps
        buffer_size=1000000,  # 1M transitions
        batch_size=32
    ):
        self.device = device
        self.n_actions = n_actions
        self.gamma = gamma
        self.epsilon_start = epsilon_start
        self.epsilon_end = epsilon_end
        self.epsilon_decay = epsilon_decay
        self.target_update_freq = target_update_freq
        self.batch_size = batch_size
        self.steps = 0
        
        # Create Q-network and target network
        self.q_network = DQNModel(n_actions).to(device)
        self.target_network = DQNModel(n_actions).to(device)
        self.target_network.load_state_dict(self.q_network.state_dict())
        self.target_network.eval()
        
        # Optimizer
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=lr, eps=1.5e-4)
        
        # Replay buffer
        self.replay_buffer = ReplayBuffer(buffer_size)
    
    def get_epsilon(self):
        """Calculate current epsilon value."""
        if self.steps < self.epsilon_decay:
            return self.epsilon_end + (self.epsilon_start - self.epsilon_end) * \
                   (1 - self.steps / self.epsilon_decay)
        return self.epsilon_end
    
    def select_action(self, state, training=True):
        """Select action using epsilon-greedy policy."""
        if training and random.random() < self.get_epsilon():
            return random.randrange(self.n_actions)
        
        # Greedy action selection
        with torch.no_grad():
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(self.device)
            q_values = self.q_network(state_tensor)
            return q_values.argmax(1).item()
    
    def store_transition(self, state, action, reward, next_state, done):
        """Store transition in replay buffer."""
        self.replay_buffer.push(state, action, reward, next_state, done)
        self.steps += 1
    
    def train_step(self):
        """Perform one training step using Double DQN."""
        if len(self.replay_buffer) < self.batch_size:
            return None
        
        # Sample batch
        states, actions, rewards, next_states, dones = self.replay_buffer.sample(self.batch_size)
        states = states.to(self.device)
        actions = actions.to(self.device)
        rewards = rewards.to(self.device)
        next_states = next_states.to(self.device)
        dones = dones.to(self.device)
        
        # Current Q-values
        q_values = self.q_network(states)
        q_value = q_values.gather(1, actions.unsqueeze(1)).squeeze(1)
        
        # Double DQN: use main network to select, target network to evaluate
        with torch.no_grad():
            next_q_values_main = self.q_network(next_states)
            next_actions = next_q_values_main.argmax(1)
            next_q_values_target = self.target_network(next_states)
            next_q_value = next_q_values_target.gather(1, next_actions.unsqueeze(1)).squeeze(1)
            target_q_value = rewards + (1 - dones) * self.gamma * next_q_value
        
        # Compute loss
        loss = F.mse_loss(q_value, target_q_value)
        
        # Optimize
        self.optimizer.zero_grad()
        loss.backward()
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(self.q_network.parameters(), 10)
        self.optimizer.step()
        
        # Update target network
        if self.steps % self.target_update_freq == 0:
            self.target_network.load_state_dict(self.q_network.state_dict())
        
        return loss.item()
    
    def save(self, filepath):
        """Save model checkpoint."""
        torch.save({
            'q_network': self.q_network.state_dict(),
            'target_network': self.target_network.state_dict(),
            'optimizer': self.optimizer.state_dict(),
            'steps': self.steps,
        }, filepath)
    
    def load(self, filepath):
        """Load model checkpoint."""
        checkpoint = torch.load(filepath, map_location=self.device)
        self.q_network.load_state_dict(checkpoint['q_network'])
        self.target_network.load_state_dict(checkpoint['target_network'])
        self.optimizer.load_state_dict(checkpoint['optimizer'])
        self.steps = checkpoint['steps']


print("✅ DQN Agent defined")

✅ DQN Agent defined


In [6]:
# Cell 6: Training Loop

# Create environment
env = make_atari_env('PongNoFrameskip-v4')
state_shape = env.observation_space.shape
n_actions = env.action_space.n

print(f"Environment: PongNoFrameskip-v4")
print(f"State shape: {state_shape}")
print(f"Number of actions: {n_actions}")

# Create agent
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

agent = DQNAgent(
    state_shape=state_shape,
    n_actions=n_actions,
    device=device,
    lr=0.0000625,
    gamma=0.99,
    epsilon_start=1.0,
    epsilon_end=0.1,
    epsilon_decay=1000000,
    target_update_freq=10000,
    buffer_size=1000000,
    batch_size=32
)

# Training parameters
total_steps = 10_000_000  # 10M steps
learning_starts = 50_000  # Start learning after 50k steps
train_freq = 4  # Train every 4 steps
save_freq = 500_000  # Save every 500k steps
eval_freq = 100_000  # Evaluate every 100k steps

# Statistics
episode_rewards = []
episode_lengths = []
current_episode_reward = 0
current_episode_length = 0

# Create models directory
os.makedirs("models", exist_ok=True)
model_path = "models/dqn_pong.pth"

# Initialize environment
state, info = env.reset()
print(f"\n✅ Starting training from step 0")
print(f"   Total steps: {total_steps:,}")
print(f"   Learning starts at: {learning_starts:,} steps")

# Training loop
pbar = tqdm(range(total_steps), desc="Training")

for step in pbar:
    # Select action
    action = agent.select_action(state, training=True)
    
    # Take step
    next_state, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated
    
    # Store transition
    agent.store_transition(state, action, reward, next_state, done)
    
    # Update statistics
    current_episode_reward += reward
    current_episode_length += 1
    
    # Train
    if step >= learning_starts and step % train_freq == 0:
        loss = agent.train_step()
        if loss is not None:
            pbar.set_postfix({
                'epsilon': f'{agent.get_epsilon():.3f}',
                'loss': f'{loss:.4f}',
                'avg_reward': f'{np.mean(episode_rewards[-10:]):.1f}' if len(episode_rewards) >= 10 else 'N/A'
            })
    
    # Handle episode end
    if done:
        episode_rewards.append(current_episode_reward)
        episode_lengths.append(current_episode_length)
        current_episode_reward = 0
        current_episode_length = 0
        state, info = env.reset()
    else:
        state = next_state
    
    # Save model periodically
    if step > 0 and step % save_freq == 0:
        agent.save(model_path)
        print(f"\n💾 Model saved at step {step:,}")
        if len(episode_rewards) >= 10:
            recent_avg = np.mean(episode_rewards[-10:])
            print(f"   Recent avg reward: {recent_avg:.2f}")
    
    # Evaluate and show progress
    if step > 0 and step % eval_freq == 0 and len(episode_rewards) >= 10:
        avg_reward = np.mean(episode_rewards[-10:])
        avg_length = np.mean(episode_lengths[-10:])
        print(f"\n📊 Step {step:,}:")
        print(f"   Avg reward (last 10): {avg_reward:.2f}")
        print(f"   Avg episode length: {avg_length:.1f}")
        print(f"   Epsilon: {agent.get_epsilon():.3f}")
        
        # Progress indicators
        if avg_reward > 10:
            print("   🎉🎉🎉 EXCELLENT! Agent is winning!")
        elif avg_reward > 0:
            print("   🎉 BREAKTHROUGH! Agent is winning!")
        elif avg_reward > -10:
            print("   🎯 Great progress! Agent is learning!")
        elif avg_reward > -15:
            print("   📈 Starting to improve!")
        elif avg_reward > -19:
            print("   📊 Better than random!")
        else:
            print("   ⏳ Still exploring...")
    
    pbar.update(1)

# Final save
agent.save(model_path)
env.close()
pbar.close()

print(f"\n✅ Training complete!")
print(f"Total episodes: {len(episode_rewards)}")
if len(episode_rewards) > 0:
    print(f"Final average reward (last 100): {np.mean(episode_rewards[-100:]):.2f}")
    print(f"Best average reward (last 10): {np.mean(episode_rewards[-10:]):.2f}")

NameNotFound: Environment `PongNoFrameskip` doesn't exist.